# Project 3: Text classification

In this assignment, we will be using Neural Networks to classify movie reviews as positive or negative. This is commonly referred to as "sentiment analysis". While the actual classification might be unique to different tasks you might encounter, the first few steps to setting up a recurrent neural net (just as the steps to building a convolutional neural network) are common to all RNN structures.

Goals:
- Understand recurrent neural networks
- Understand an end to end NN
- Implement a recurrent neural network

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
from keras.datasets import imdb
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

C:\Users\furon\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Download the data if you don't already have it. 
# Otherwise, keras will load it in.
# The data will be saved in ~/.keras/datasets
dim = 300

(x, y), _ = imdb.load_data(num_words=5000)
y_vectorized = to_categorical(y)

In [4]:
words = imdb.get_word_index()

## Word embeddings

Words cannot be simply passed into a neural network. There are several common approaches for turning words into a format neural networks can understand, the most useful of which is associating each word with an "embedding". That is, representing the row as a vector of numbers (that can represent arbitrary features). You are free to train your own word embeddings, but it is common to simply take ones generated from huge text corpora.

We will be using the GloVe word embeddings in this exercise, as these vectors have been trained from over 6 billion English language tokens. They are free to download.

In [5]:
from exercise_3 import create_word_embeddings
try:
    with np.load(
        os.path.join("glove.6B.{}d.trimmed.npz".format(dim))) as data:
        embeddings = data["embeddings"]
except FileNotFoundError:
    embeddings = create_word_embeddings("glove", words, save=True)

## Sequence Padding

Neural networks tend not to handle inputs of different sizes very well. So, with sequences such as sentences or timeseries of varying length, the common approach is to "pad" the sequence with 0's on the end so that all inputs are the same length. 

In [6]:
from keras.preprocessing.sequence import pad_sequences

In [7]:
pad_sequences(x[:10]).shape

(10, 562)

## Exercise 1:

As always, we want to build a dense (fully-connected) model as our baseline. However, this is tricky because the number of units in fully connected layers is hard-coded. So, to pass a sequence into this you need to reshape your embedded words into the right shape. Practice this by implementing build_dense_model() in exercise_3.py.

In [10]:
from exercise_3 import build_dense_model

In [11]:
dense_max_sequence_length = max([len(xi) for xi in x])

In [12]:
dense_model = build_dense_model(embeddings)

In [13]:
# make sure your model is predicting the correct dimension:
example_prediction = dense_model.predict(pad_sequences(x[:10], 2494))
assert example_prediction.shape == (10,2)

In [12]:
# this will run model fitting and validation on the dev set
dense_model.fit(pad_sequences(x[:10], 2494), y_vectorized[:10],
                validation_data=(pad_sequences(x[:10], 2494), y_vectorized[:10]),
                batch_size=2, epochs=10, 
                verbose=1)

Train on 10 samples, validate on 10 samples
Epoch 1/10
10/10 [==============================] - 69s 7s/step - loss: 1.6511 - acc: 0.3000 - val_loss: 3.2236 - val_acc: 0.8000
Epoch 2/10
10/10 [==============================] - 47s 5s/step - loss: 1.6832 - acc: 0.3000 - val_loss: 1.5076 - val_acc: 0.7000
Epoch 3/10
10/10 [==============================] - 45s 4s/step - loss: 1.4523 - acc: 0.3000 - val_loss: 0.8255 - val_acc: 0.6000
Epoch 4/10
10/10 [==============================] - 36s 4s/step - loss: 1.0162 - acc: 0.4000 - val_loss: 0.4306 - val_acc: 0.7000
Epoch 5/10
10/10 [==============================] - 33s 3s/step - loss: 0.4555 - acc: 0.8000 - val_loss: 0.3683 - val_acc: 0.8000
Epoch 6/10
10/10 [==============================] - 36s 4s/step - loss: 0.8802 - acc: 0.5000 - val_loss: 0.3727 - val_acc: 0.8000
Epoch 7/10
10/10 [==============================] - 36s 4s/step - loss: 0.2544 - acc: 0.9000 - val_loss: 0.3686 - val_acc: 0.8000
Epoch 8/10
10/10 [============================

## Exercise 2:

Clearly, the fully connected approach is not adequate for sequence modeling, especially if we run the risk of a future sentence being longer than our maximally allocated sentence length. Furthermore, if a few sentences in our dataset are especially long and most are short, this leads to a lot of unnecessary computation.

Enter the Recurrent neural network architecture (specifically, simpleRNN).

In [14]:
from exercise_3 import build_simpleRNN_model

In [15]:
simpleRNN_model = build_simpleRNN_model(embeddings)

In [16]:
# this will run model fitting and validation on the dev set
simpleRNN_model.fit(pad_sequences(x[:10]), y_vectorized[:10],
                validation_data=(pad_sequences(x[:10]), y_vectorized[:10]),
                batch_size=2, epochs=5, 
                verbose=1)

Train on 10 samples, validate on 10 samples
Epoch 1/5
10/10 [==============================] - 1s 78ms/step - loss: 0.8949 - acc: 0.5000 - val_loss: 0.0183 - val_acc: 1.0000
Epoch 2/5
10/10 [==============================] - 0s 41ms/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 3/5
10/10 [==============================] - 0s 49ms/step - loss: 0.0038 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 4/5
10/10 [==============================] - 0s 47ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 5/5
10/10 [==============================] - 0s 46ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000


## Exercise 3:

Complete build_lstm_model() in exercise_3.py 

In [17]:
from exercise_3 import build_lstm_model

In [18]:
lstm_model = build_lstm_model(embeddings)

In [19]:
# this will run model fitting and validation on the dev set
lstm_model.fit(pad_sequences(x[:10]), y_vectorized[:10],
                validation_data=(pad_sequences(x[:10]), y_vectorized[:10]),
                batch_size=2, epochs=5, 
                verbose=1)

Train on 10 samples, validate on 10 samples
Epoch 1/5
10/10 [==============================] - 2s 250ms/step - loss: 0.9866 - acc: 0.4000 - val_loss: 0.3801 - val_acc: 1.0000
Epoch 2/5
10/10 [==============================] - 2s 155ms/step - loss: 0.3983 - acc: 0.8000 - val_loss: 0.2347 - val_acc: 1.0000
Epoch 3/5
10/10 [==============================] - 2s 167ms/step - loss: 0.1988 - acc: 1.0000 - val_loss: 0.0841 - val_acc: 1.0000
Epoch 4/5
10/10 [==============================] - 2s 158ms/step - loss: 0.0489 - acc: 1.0000 - val_loss: 0.0218 - val_acc: 1.0000
Epoch 5/5
10/10 [==============================] - 2s 152ms/step - loss: 0.0132 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 1.0000


## Exercise 4: 

In [20]:
from exercise_3 import build_bidirectional_lstm_model

In [21]:
bi_lstm_model = build_bidirectional_lstm_model(embeddings)
# this will run model fitting and validation on the dev set
bi_lstm_model.fit(pad_sequences(x[:10]), y_vectorized[:10],
                validation_data=(pad_sequences(x[:10]), y_vectorized[:10]),
                batch_size=2, epochs=5, 
                verbose=1)

Train on 10 samples, validate on 10 samples
Epoch 1/5
10/10 [==============================] - 3s 341ms/step - loss: 0.9880 - acc: 0.3000 - val_loss: 0.3368 - val_acc: 0.9000
Epoch 2/5
10/10 [==============================] - 3s 272ms/step - loss: 0.2817 - acc: 1.0000 - val_loss: 0.1792 - val_acc: 1.0000
Epoch 3/5
10/10 [==============================] - 3s 260ms/step - loss: 0.1407 - acc: 1.0000 - val_loss: 0.0618 - val_acc: 1.0000
Epoch 4/5
10/10 [==============================] - 3s 257ms/step - loss: 0.0433 - acc: 1.0000 - val_loss: 0.0106 - val_acc: 1.0000
Epoch 5/5
10/10 [==============================] - 3s 308ms/step - loss: 0.1317 - acc: 0.9000 - val_loss: 0.0036 - val_acc: 1.0000


## Exercise 5:

You should notice that the bidirectional LSTM can perfectly overfit the data in under 5 epochs. Let's fit the full model using it. Implement build_final_model() in exercise_3.py to get the best review classification as you can using any RNN techniques discussed in class. 

In [22]:
x_train, x_dev, y_train, y_dev = train_test_split(x, y_vectorized)

In [33]:
from exercise_3 import build_final_model

In [34]:
final_model = build_final_model(embeddings)
# this will run model fitting and validation on the dev set
final_model.fit(pad_sequences(x_train), y_train,
                validation_data=(pad_sequences(x_dev), y_dev),
                batch_size=256, epochs=1, 
                verbose=1)

Train on 18750 samples, validate on 6250 samples
Epoch 1/1
18750/18750 [==============================] - 54123s 3s/step - loss: 0.6933 - acc: 0.5244 - val_loss: 0.6788 - val_acc: 0.5656


In [35]:
x_test, y_test = _
y_test = to_categorical(y_test)

In [ ]:
from sklearn.metrics import classification_report
predicted = final_model.predict_classes(pad_sequences(x_test, 2494))
classification_report(y_test,to_categorical(predicted))

## Exercise 6:
Consider your experience implementing a simple RNN, LSTM and BiLSTM. What advantages and disadvantages do you notice?

(double click to edit) Or submit a separate file with your write up.

### RNNs: 

###### Advantages:  
Compared to feed forward neural networks, RNN is be able to connect previous information to the present task, it takes into consideration the current input and also what it has learned from the inputs it received previously. Therefore RNN has better performance for sequential data like time series, speech, text, financial data, audio, video and weather, etc. 

###### Disadvantages: 
Hochreiter’s 1991 MSc thesis (in German) had independently discovered that backpropagated gradients in RNNs tend to either vanish or explode as sequence length increases. RNN is able to remember only short-term memory sequences and gives Long-term dependencies weights that are exponen1allysmalle r (in T) compared to short-term dependencies. Therefoe, as the time steps increase, it fails to derive context from time steps which are much far behind. 

### LSTM: 
###### Advantages:  
The vanishing gredient problem is addressed in LSTM (Long Short Term Memory) model, which is capable of learning long-term dependencies. Therefore it is well suited to learn from important experiences that have very long time lags in between. LSTM’s enable RNN’s to remember their inputs over a long period of time. This is because LSTM’s contain their information in a memory, that is much like the memory of a computer because the LSTM can read, write and delete information from its memory.

###### Disadvantages:
A major issue with LSTM is that they learn representations from previous time steps. Sometimes, one might have to learn representations from future time steps to better understand the context and eliminate ambiguity. 

### Bidrectional LSTM: 
###### Advantages:  
Using bidirectional will run the inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backwards you preserve information from the future and using the two hidden states combined you are able in any point in time to preserve information from both past and future.

###### Disadvantages:
A lot of time and system resources go into training even a simple model. But that is just a hardware constraint.

References:
https://machinelearning-blog.com/2018/02/21/recurrent-neural-networks/
https://drive.google.com/file/d/0ByUKRdiCDK7-LXZkM3hVSzFGTkE/view
https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/
http://colah.github.io/posts/2015-08-Understanding-LSTMs/